## 모델링

## 라이브러리 설치 및 버전 확인

In [5]:
# 라이브러리 설치
!pip install pandas
!pip install numpy
!pip install tensorflow
!pip install keras_tuner
!pip install sklearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 6.1 MB/s eta 0:00:00


In [6]:
# 라이브러리 import
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt 
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import __version__ as sklearn_version
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.metrics import f1_score, roc_auc_score, precision_score
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import AUC
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score, roc_auc_score, recall_score

In [7]:
# 라이브러리 버전출력
print("Pandas version: ", pd.__version__)
print("Numpy version: ", np.__version__)
print("TensorFlow version: ", tf.__version__)
print("Keras Tuner version: ", kt.__version__)
print("Scikit-learn version: ", sklearn_version)

Pandas version:  2.0.2
Numpy version:  1.24.3
TensorFlow version:  2.13.0
Keras Tuner version:  1.3.5
Scikit-learn version:  1.2.2


## 데이터 불러오기 및 target값 만들기

In [8]:
# 학습용 데이터 불러오기
data = pd.read_csv("/project/datacamp/team5/data/scale_all_data_9~0.csv")
data


,code,date,Open_D-0,High_D-0,Low_D-0,Close_D-0,Trading_D-0,capitalization_D-0,MA5_D-0,MA20_D-0,...,monthly_Open_D-9,monthly_High_D-9,monthly_Low_D-9,monthly_Close_D-9,monthly_Trading_D-9,monthly_capitalization_D-9,M_MA5_D-9,M_MA10_D-9,M_MA20_D-9,next_change
0,5930,2013-01-15,0.695721,0.695721,0.680161,0.680177,16.172016,22.693889,0.684541,0.680568,...,0.696754,0.710715,0.694788,0.710731,16.274599,22.754865,0.634569,0.616802,0.550782,-0.013228
1,5930,2013-01-16,0.693808,0.697750,0.683511,0.686511,16.595346,22.706685,0.697042,0.692866,...,0.679411,0.695682,0.677479,0.682622,17.048322,22.698839,0.619191,0.601413,0.536781,-0.015416
2,5930,2013-01-17,0.696493,0.696493,0.678972,0.685410,16.806491,22.704465,0.699694,0.698778,...,0.689901,0.706346,0.682380,0.687297,17.471375,22.708266,0.629723,0.611516,0.546220,0.007488
3,5930,2013-01-18,0.696867,0.699238,0.688354,0.696884,16.345940,22.727461,0.703980,0.706233,...,0.695767,0.712307,0.684900,0.691506,17.760619,22.716716,0.635848,0.617400,0.551776,-0.017568
4,5930,2013-01-21,0.687727,0.691457,0.682260,0.684325,16.476335,22.702277,0.693620,0.701440,...,0.697414,0.713981,0.685884,0.686546,17.998214,22.706755,0.637923,0.619340,0.553736,0.017882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356428,57050,2023-06-26,0.693147,0.698069,0.692050,0.693695,8.113357,16.301513,0.701990,0.709811,...,0.679412,0.703586,0.676749,0.696813,11.612374,16.307736,0.721493,0.717837,0.762730,0.008762
356429,57050,2023-06-27,0.691503,0.698064,0.690954,0.697519,8.385305,16.309141,0.699806,0.708041,...,0.675810,0.699895,0.673157,0.694708,11.695535,16.303537,0.717032,0.713971,0.758293,0.000000
356430,57050,2023-06-28,0.689885,0.694775,0.688249,0.693147,8.942275,16.300417,0.693039,0.702764,...,0.674279,0.698326,0.671630,0.688985,11.763781,16.292075,0.714801,0.712250,0.756072,-0.016287
356431,57050,2023-06-29,0.689885,0.695858,0.684971,0.684971,8.797550,16.283997,0.689667,0.702119,...,0.678379,0.702527,0.675719,0.683678,11.859351,16.281389,0.718248,0.716342,0.759932,0.011038


In [9]:
# 'Next Change' 값에 따라 'target' 컬럼 생성
data['target'] = data['next_change'].apply(lambda x: 1 if x >= 0.05 else 0)
data

,code,date,Open_D-0,High_D-0,Low_D-0,Close_D-0,Trading_D-0,capitalization_D-0,MA5_D-0,MA20_D-0,...,monthly_High_D-9,monthly_Low_D-9,monthly_Close_D-9,monthly_Trading_D-9,monthly_capitalization_D-9,M_MA5_D-9,M_MA10_D-9,M_MA20_D-9,next_change,target
0,5930,2013-01-15,0.695721,0.695721,0.680161,0.680177,16.172016,22.693889,0.684541,0.680568,...,0.710715,0.694788,0.710731,16.274599,22.754865,0.634569,0.616802,0.550782,-0.013228,0
1,5930,2013-01-16,0.693808,0.697750,0.683511,0.686511,16.595346,22.706685,0.697042,0.692866,...,0.695682,0.677479,0.682622,17.048322,22.698839,0.619191,0.601413,0.536781,-0.015416,0
2,5930,2013-01-17,0.696493,0.696493,0.678972,0.685410,16.806491,22.704465,0.699694,0.698778,...,0.706346,0.682380,0.687297,17.471375,22.708266,0.629723,0.611516,0.546220,0.007488,0
3,5930,2013-01-18,0.696867,0.699238,0.688354,0.696884,16.345940,22.727461,0.703980,0.706233,...,0.712307,0.684900,0.691506,17.760619,22.716716,0.635848,0.617400,0.551776,-0.017568,0
4,5930,2013-01-21,0.687727,0.691457,0.682260,0.684325,16.476335,22.702277,0.693620,0.701440,...,0.713981,0.685884,0.686546,17.998214,22.706755,0.637923,0.619340,0.553736,0.017882,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356428,57050,2023-06-26,0.693147,0.698069,0.692050,0.693695,8.113357,16.301513,0.701990,0.709811,...,0.703586,0.676749,0.696813,11.612374,16.307736,0.721493,0.717837,0.762730,0.008762,0
356429,57050,2023-06-27,0.691503,0.698064,0.690954,0.697519,8.385305,16.309141,0.699806,0.708041,...,0.699895,0.673157,0.694708,11.695535,16.303537,0.717032,0.713971,0.758293,0.000000,0
356430,57050,2023-06-28,0.689885,0.694775,0.688249,0.693147,8.942275,16.300417,0.693039,0.702764,...,0.698326,0.671630,0.688985,11.763781,16.292075,0.714801,0.712250,0.756072,-0.016287,0
356431,57050,2023-06-29,0.689885,0.695858,0.684971,0.684971,8.797550,16.283997,0.689667,0.702119,...,0.702527,0.675719,0.683678,11.859351,16.281389,0.718248,0.716342,0.759932,0.011038,0


In [10]:
data = data.drop("next_change", axis=1)

In [11]:
# "date_D0" 컬럼을 datetime 형태로 변환
data["date"] = pd.to_datetime(data["date"])

In [12]:
train = data[data['date'] < '2020-07-01']
val = data[(data['date'] >= '2020-07-01') & (data['date'] < '2022-01-01')]
test = data[data['date'] >= '2022-01-01']
print(train.shape, val.shape, test.shape)

(253548, 733) (51654, 733) (51231, 733)


## 학습

In [13]:
# 범위 지정
temp_train = train[train.columns[2:-1]]
trainY = train['target']
temp_train

,Open_D-0,High_D-0,Low_D-0,Close_D-0,Trading_D-0,capitalization_D-0,MA5_D-0,MA20_D-0,MA60_D-0,MA120_D-0,...,W_MA20_D-9,monthly_Open_D-9,monthly_High_D-9,monthly_Low_D-9,monthly_Close_D-9,monthly_Trading_D-9,monthly_capitalization_D-9,M_MA5_D-9,M_MA10_D-9,M_MA20_D-9
0,0.695721,0.695721,0.680161,0.680177,16.172016,22.693889,0.684541,0.680568,0.649096,0.626370,...,0.634569,0.696754,0.710715,0.694788,0.710731,16.274599,22.754865,0.634569,0.616802,0.550782
1,0.693808,0.697750,0.683511,0.686511,16.595346,22.706685,0.697042,0.692866,0.662530,0.639568,...,0.619191,0.679411,0.695682,0.677479,0.682622,17.048322,22.698839,0.619191,0.601413,0.536781
2,0.696493,0.696493,0.678972,0.685410,16.806491,22.704465,0.699694,0.698778,0.669958,0.646757,...,0.629723,0.689901,0.706346,0.682380,0.687297,17.471375,22.708266,0.629723,0.611516,0.546220
3,0.696867,0.699238,0.688354,0.696884,16.345940,22.727461,0.703980,0.706233,0.678524,0.655135,...,0.635848,0.695767,0.712307,0.684900,0.691506,17.760619,22.716716,0.635848,0.617400,0.551776
4,0.687727,0.691457,0.682260,0.684325,16.476335,22.702277,0.693620,0.701440,0.675664,0.652384,...,0.637923,0.697414,0.713981,0.685884,0.686546,17.998214,22.706755,0.637923,0.619340,0.553736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355686,0.690872,0.700694,0.687830,0.699942,9.876168,16.313962,0.700995,0.717692,0.696790,0.721786,...,0.672441,0.652036,0.708309,0.650588,0.690366,12.788640,16.294846,0.672441,0.732117,0.805162
355687,0.690153,0.690153,0.675047,0.675047,9.984644,16.263884,0.689253,0.709750,0.690865,0.713870,...,0.674795,0.654708,0.711141,0.653256,0.693147,12.904235,16.300417,0.674795,0.734219,0.807789
355688,0.694696,0.710058,0.686146,0.700100,10.037759,16.314274,0.703021,0.727886,0.710223,0.731784,...,0.674070,0.654708,0.711141,0.653256,0.656883,12.998979,16.226524,0.674070,0.733261,0.807306
355689,0.682386,0.688549,0.673069,0.680839,10.194047,16.275648,0.692535,0.719405,0.703970,0.723579,...,0.710596,0.690889,0.749428,0.689381,0.726422,13.141662,16.365895,0.710596,0.771514,0.848484


In [14]:
# LSTM학습용 데이터로 변경
trainX = []
for row in temp_train.values.tolist():
    trainX.append([])
    for i in range(0, len(row), 73):
        trainX[-1].append(row[i:i+73])
trainX = np.array(trainX)
trainY = np.array(trainY)

## 검증

In [15]:
# 범위 지정
temp_val = val[val.columns[2:-1]]
valY = val['target']

In [16]:
# LSTM학습용 데이터로 변경
valX = []
for row in temp_val.values.tolist():
    valX.append([])
    for i in range(0, len(row), 73):
        valX[-1].append(row[i:i+73])
valX = np.array(valX)
valY = np.array(valY)

## 테스트

In [17]:
# 범위 지정
temp_test = test[test.columns[2:-1]]
testY = test['target']

In [18]:
# LSTM학습용 데이터로 변경
testX = []
for row in temp_test.values.tolist():
    testX.append([])
    for i in range(0, len(row), 73):
        testX[-1].append(row[i:i+73])
testX = np.array(testX)
testY = np.array(testY)

In [19]:
# shape 점검
trainX.shape, trainY.shape, valX.shape, valY.shape, testX.shape, testY.shape

((253548, 10, 73),
 (253548,),
 (51654, 10, 73),
 (51654,),
 (51231, 10, 73),
 (51231,))

## 파일저장

In [20]:
np.save('../data/trainX', trainX)
np.save('../data/trainY', trainY)
np.save('../data/valX', valX)
np.save('../data/valY', valY)
np.save('../data/testX', testX)
np.save('../data/testY', testY)

In [21]:
trainX = np.load('../data/trainX.npy')
trainY = np.load('../data/trainY.npy')
valX = np.load('../data/valX.npy')
valY = np.load('../data/valY.npy')
testX = np.load('../data/testX.npy')
testY = np.load('../data/testY.npy')

In [24]:
class_weights = compute_class_weight('balanced', classes=[0, 1], y=trainY)
class_weights

array([ 0.5132987 , 19.29882783])

In [30]:
# LSTM 모델 구성
my_model1 = Sequential()
my_model1.add(LSTM(units=224, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
my_model1.add(LSTM(units=112, return_sequences=True))  # 두 번째 LSTM 층, return_sequences=True 설정
my_model1.add(LSTM(units=56))  # 세 번째 LSTM 층

my_model1.add(Dense(units=1, activation='sigmoid'))  # 이진 분류 문제이므로 시그모이드 활성화 함수 사용

#불균형한 data set이기에 가중치 부여
class_weights = {0: 0.5132987, 1: 19.29882783}


# 모델 컴파일
#auc-roc : 이진 분류에서 모델의 전반적인 성능을 평가하는 좋은 지표 중 하나입니다. 특히 불균형한 클래스 분포를 가진 데이터셋에서 유용합니다.
my_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name = 'val_auc')])


# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor ='val_auc', patience = 5, restore_best_weights = True)

# 모델 훈련
my_model1.fit(trainX, trainY, validation_data = (valX, valY), epochs=2, batch_size = 32, callbacks=[early_stopping], class_weight=class_weights)

Epoch 1/2
7924/7924 [==============================] - 178s 22ms/step - loss: 0.6917 - val_auc: 0.5371 - val_loss: 0.7278 - val_val_auc: 0.5848
Epoch 2/2
7924/7924 [==============================] - 162s 20ms/step - loss: 0.6864 - val_auc: 0.5650 - val_loss: 0.7248 - val_val_auc: 0.5982


In [31]:
# 모델 평가
loss, accuracy = my_model1.evaluate(testX, testY)

#테스트 데이터 예측
y_pred = my_model1.predict(testX) 

1601/1601 [==============================] - 10s 6ms/step


In [32]:
y_pred

array([[0.5508329 ],
       [0.5515669 ],
       [0.5025275 ],
       ...,
       [0.41607186],
       [0.4459101 ],
       [0.45286846]], dtype=float32)

In [33]:
#예측 확률값을 이진 분류 결과로 변환
y_pred_binary = (y_pred > 0.5).astype(int)

#정확도, auc 점수 계산
f1 = f1_score(testY, y_pred_binary)
auc = roc_auc_score(testY, y_pred_binary.round())  # AUC 계산
recall = recall_score(testY, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")


Accuracy: 0.5557
AUC: 0.5353
